In [1]:
import requests
import pandas as pd
import time
from datetime import datetime
import os
import numpy as np
from tqdm import tqdm
import ta 

# Tiingo API
API_TOKEN = '4f730c224705f43752f4c8bdf20cfe9fb56ce8c5'
BASE_URL = 'https://api.tiingo.com/tiingo/daily'

STOCKS = [
    'AAPL',   # Apple - Tech
    'HSBC',   # HSBC - Finance
    'PEP',    # Pepsi - Consumer
    'TM',     # Toyota - Automotive
    'TCEHY',  # Tencent - Tech
]

START_DATE = '2015-06-20'
END_DATE = '2025-06-20'

current_dir = os.getcwd()
if os.path.basename(current_dir) == 'notebooks':
    OUTPUT_DIR = os.path.join(os.path.dirname(current_dir), 'data')
else:
    OUTPUT_DIR = os.path.join(current_dir, 'data')
os.makedirs(OUTPUT_DIR, exist_ok=True)

In [2]:
def fetch_stock_data(ticker, start_date, end_date, api_token, base_url):
    """Fetch historical stock prices from Tiingo API (OHLCV only)"""
    headers = {'Content-Type': 'application/json'}
    url = f'{base_url}/{ticker}/prices?startDate={start_date}&endDate={end_date}&token={api_token}'
    
    try:
        response = requests.get(url, headers=headers, timeout=30)
        response.raise_for_status()
        data = response.json()
        
        if data:
            df = pd.DataFrame(data)
            df['ticker'] = ticker
            df['date'] = pd.to_datetime(df['date']).dt.tz_localize(None)
            df.columns = [c.lower() for c in df.columns]
            df = df[['date', 'open', 'high', 'low', 'close', 'volume', 'ticker']].copy()
            df = df.sort_values('date').reset_index(drop=True)
            df = df.drop_duplicates(subset=['date']).reset_index(drop=True)
            return df
        return None
    except Exception as e:
        print(f"  Error fetching {ticker}: {e}")
        return None


# Data Cleaning
def clean_data(df):
    """
    Clean and normalize stock data.
    - Forward-fill volume only if needed (very limited)
    - Remove duplicates
    - Ensure date continuity
    """
    df = df.copy()
    if df['volume'].isna().any():
        df['volume'] = df['volume'].fillna(method='ffill', limit=1)
    df = df.dropna(subset=['open', 'high', 'low', 'close', 'volume'])
    return df


# Calculate Technical Indicators
def add_technical_indicators(df):
    """
    Add technical indicators to a single stock's dataframe.
    All features at time t use data <= t-1.
    Strategy: Calculate indicators, then shift by 1 period.
    Features generated:
    - Returns: return_1d, return_5d, log_return
    - Trend/Momentum: SMA_10, SMA_20, EMA_10, momentum_5, momentum_10, price_dev_SMA_20
    - Volatility: rolling_std_5, rolling_std_20, ATR_14
    - Volume: Volume_MA_5, Volume_ratio, OBV
    """
    df = df.copy()
    
    # Returns
    df['return_1d'] = df['close'].pct_change()
    df['return_5d'] = df['close'].pct_change(5)
    df['log_return'] = np.log(df['close'] / df['close'].shift(1))
    
    # Moving Averages
    df['SMA_10'] = ta.trend.sma_indicator(df['close'], window=10)
    df['SMA_20'] = ta.trend.sma_indicator(df['close'], window=20)
    df['EMA_10'] = ta.trend.ema_indicator(df['close'], window=10)
    
    # Price deviation from SMA_20
    df['price_dev_SMA_20'] = (df['close'] - df['SMA_20']) / (df['SMA_20'] + 1e-8)
    
    # Momentum
    df['momentum_5'] = df['close'] - df['close'].shift(5)
    df['momentum_10'] = df['close'] - df['close'].shift(10)
    
    # RSI
    df['RSI_14'] = ta.momentum.rsi(df['close'], window=14)
    
    # MACD
    macd = ta.trend.MACD(df['close'])
    df['MACD'] = macd.macd()
    df['MACD_signal'] = macd.macd_signal()
    df['MACD_diff'] = macd.macd_diff()
    
    # Bollinger Bands
    bb = ta.volatility.BollingerBands(df['close'], window=20)
    df['BB_upper'] = bb.bollinger_hband()
    df['BB_lower'] = bb.bollinger_lband()
    df['BB_middle'] = bb.bollinger_mavg()
    df['BB_width'] = (df['BB_upper'] - df['BB_lower']) / (df['BB_middle'] + 1e-8)
    
    # Volatility
    df['rolling_std_5'] = df['close'].pct_change().rolling(5).std()
    df['rolling_std_20'] = df['close'].pct_change().rolling(20).std()
    df['ATR_14'] = ta.volatility.average_true_range(df['high'], df['low'], df['close'], window=14)
    
    # Volume indicators
    df['Volume_MA_5'] = df['volume'].rolling(5).mean()
    df['Volume_ratio'] = df['volume'] / (df['Volume_MA_5'] + 1e-8)
    df['OBV'] = ta.volume.on_balance_volume(df['close'], df['volume'])
    
    tech_cols = [
        # Returns
        'return_1d', 'return_5d', 'log_return',
        # Trend/Momentum
        'SMA_10', 'SMA_20', 'EMA_10', 'price_dev_SMA_20', 'momentum_5', 'momentum_10',
        # RSI, MACD
        'RSI_14', 'MACD', 'MACD_signal', 'MACD_diff',
        # Bollinger Bands
        'BB_upper', 'BB_lower', 'BB_middle', 'BB_width',
        # Volatility
        'rolling_std_5', 'rolling_std_20', 'ATR_14',
        # Volume
        'Volume_MA_5', 'Volume_ratio', 'OBV'
    ]
    
    for col in tech_cols:
        if col in df.columns:
            df[col] = df[col].shift(1)
    
    return df


def process_single_stock(ticker, start_date, end_date, api_token, base_url):
    """
    Complete pipeline for a single stock (OHLCV-only):
    1. Fetch stock prices from Tiingo API
    2. Clean data
    3. Calculate technical indicators
    """
    stock_df = fetch_stock_data(ticker, start_date, end_date, api_token, base_url)
    if stock_df is None:
        print(f"  Failed to fetch stock data for {ticker}")
        return None
    
    print(f"  Stock data: {len(stock_df)} records")
    
    stock_df = clean_data(stock_df)
    print(f"  After cleaning: {len(stock_df)} records")
    
    stock_df = add_technical_indicators(stock_df)
    
    initial_len = len(stock_df)
    stock_df = stock_df.dropna().reset_index(drop=True)
    print(f"  Final: {len(stock_df)} records (dropped {initial_len - len(stock_df)} rows with NaN)")
    
    return stock_df

In [3]:
def process_all_stocks(stocks, start_date, end_date, api_token, base_url, output_dir):
    print("="*70)
    print(f"Stocks: {stocks}")
    print(f"Period: {start_date} to {end_date}")
    print(f"Output: {output_dir}/{{TICKER}}.csv")
    print("="*70)
    
    results = {}
    
    for i, ticker in enumerate(stocks, 1):
        print(f"\n[{i}/{len(stocks)}]", end="")
        
        df = process_single_stock(
            ticker, start_date, end_date, 
            api_token, base_url
        )
        
        if df is not None:
            output_file = f'{output_dir}/{ticker}.csv'
            df.to_csv(output_file, index=False)
            
            results[ticker] = {
                'records': len(df),
                'columns': len(df.columns),
                'date_range': f"{df['date'].min()} to {df['date'].max()}",
            }
            print(f"  Saved: {output_file}")
        else:
            results[ticker] = None
        time.sleep(1.0)
    return results

results = process_all_stocks(
    STOCKS, START_DATE, END_DATE, 
    API_TOKEN, BASE_URL, OUTPUT_DIR
)


Stocks: ['AAPL', 'HSBC', 'PEP', 'TM', 'TCEHY']
Period: 2015-06-20 to 2025-06-20
Output: c:\Users\minhq\Documents\Projects\stocks-trend-prediction\data/{TICKER}.csv

[1/5]  Stock data: 2515 records
  After cleaning: 2515 records
  Final: 2481 records (dropped 34 rows with NaN)
  Saved: c:\Users\minhq\Documents\Projects\stocks-trend-prediction\data/AAPL.csv

[2/5]  Stock data: 2515 records
  After cleaning: 2515 records
  Final: 2481 records (dropped 34 rows with NaN)
  Saved: c:\Users\minhq\Documents\Projects\stocks-trend-prediction\data/HSBC.csv

[3/5]  Stock data: 2515 records
  After cleaning: 2515 records
  Final: 2481 records (dropped 34 rows with NaN)
  Saved: c:\Users\minhq\Documents\Projects\stocks-trend-prediction\data/PEP.csv

[4/5]  Stock data: 2515 records
  After cleaning: 2515 records
  Final: 2481 records (dropped 34 rows with NaN)
  Saved: c:\Users\minhq\Documents\Projects\stocks-trend-prediction\data/TM.csv

[5/5]  Stock data: 2515 records
  After cleaning: 2515 records

### Summary

In [4]:
for ticker, info in results.items():
    if info:
        print(f"\n{ticker}:")
        print(f"  Records: {info['records']} | Columns: {info['columns']}")
        print(f"  Date Range: {info['date_range']}")
    else:
        print(f"\n{ticker}: FAILED")



AAPL:
  Records: 2481 | Columns: 30
  Date Range: 2015-08-10 00:00:00 to 2025-06-20 00:00:00

HSBC:
  Records: 2481 | Columns: 30
  Date Range: 2015-08-10 00:00:00 to 2025-06-20 00:00:00

PEP:
  Records: 2481 | Columns: 30
  Date Range: 2015-08-10 00:00:00 to 2025-06-20 00:00:00

TM:
  Records: 2481 | Columns: 30
  Date Range: 2015-08-10 00:00:00 to 2025-06-20 00:00:00

TCEHY:
  Records: 2481 | Columns: 30
  Date Range: 2015-08-10 00:00:00 to 2025-06-20 00:00:00
